The purpose of this notebook is to build a robust cluster finding algorithm that produces meaningful results for my computational astrophysics star cluster formation simulations. Typical out-of-the-box cluster finders rely on particle number density, or nearest neighbor criteria. Such techniques work well for observational research in which less information is known about star or galaxy cluster members. 

What am I interested in?
* Analysis on how much material (stars) is in a bound state within the identified clusters
* use 'boundedness' as a criteria for cluster identification
* fraction of bound stars in system over time

What scientific question am I trying to answer with this tool?
* Early forming massive stars form looser assosciations of stars rather than allowing for more massive clusters.
    * potential profile of cluster
    * density profile of cluster

    
Methods to create this tool:
* Have DBSCAN find clusters from all stars and report.
* remove stars that have negative energy (considering all other stars and gas). Recalculate star total energies. Remove E_tot > 0 stars, recalculate energies. Repeat until all stars have E_tot < 0. 
    * This method feels like it might have some issues as the removed stars still will contribute to potential in reality.
    * but [Li+2019](https://arxiv.org/pdf/1904.11987.pdf) found that such a recursive technique results in identification of clusters with lagrangiian radii consistently near the 60% mass bin using the conventional cluster identification method.

# Vanilla DBSCAN technique
DBSCAN ([Wiki](https://en.wikipedia.org/wiki/DBSCAN), [scipy docs](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.DBSCAN.html)) or Density-Based Spatial Clustering of Applications with Noise, is a clustering algorithm originally proposed in 1996 (Ester et al. 1996) which idenitifies data points that are densely clustered (points with many nearest neighbors) and excludes those with few nearest neighbors.

DBSCAN can be used to identify star clusters. 
1. Input all stars in computational domain
2. Pass through DBSCAN, report clusters

Cons to this method: DBSCAN does not have any knowledge of dynamics. Only looks for spatially separated density peaks, therefore will have a tendency to overestimate the fraction of stars that are truly members of a particular cluster (includes stars in a cluster even if the stars have v > escape velocity).

In [1]:
import numpy as np
from amuse.lab import generic_unit_converter, nbody_system
from amuse.lab import units as u
from amuse.lab import Particles
import amuse.lab

import matplotlib
matplotlib.use('Agg')
import matplotlib
font = {'family' : 'sans',
        'weight' : 'normal',
        'size'   : 24}

matplotlib.rc('font', **font)
matplotlib.rc({'savefig.dpi':300})
import matplotlib.pyplot as plt

from functools import partial
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import DBSCAN

import sys
import h5py

In [36]:
def find_clusters_with_dbscan(stars, outer_density_limit=1.0 | u.MSun*u.parsec**-3,
                              avg_stellar_mass=0.586,
                              eps=0.4, min_samples=12, leaf_size=30,
                              return_labels=False, debug=False):

    """
    Find all the stars in clusters using
    the DBSCAN implementation from scikit-learn.

    Keyword Arguments:
    stars               -- AMUSE particle set
    outer_density_limit -- If set, use this density_limit
                           in solar masses per parsec^-3 to
                           compute eps to figure
                           out where the cluster edges are
                           instead of the input eps. 
                           A good default choice
                           is 1.0 MSun * pc^-3.
                           Note this setting overrides eps.
    avg_stellar_mass    -- Average stellar mass of the IMF
                           used to make the stars your
                           clustering. Default is 
                           from a Kroupa IMF that goes 
                           from 0.08 to 150 Msun.
    eps                 -- Minimum  neighbor distance to be
                           considered in the cluster in pc.
                           This value is calculated for you
                           if you use outer_density_limit.
    min_samples         -- Minimum number of neighbors to
                           be considered a core particle.
                           Default is 12, the default for
                           DBSCAN.
    leaf_size           -- Number of particles in a leaf
                           on the KD tree the code uses
                           to find neighbors. Default is
                           30, the default for DBSCAN.
    return_labels       -- Also return the raw DBSCAN label output?
    debug               -- Turn on debugging output

    Returns:
    groups        -- A list of particle sets for the particles in each cluster.
    n_groups      -- The number of clusters found.
    labels        -- The actual label indicies returned by DBSCAN,
                     only returned if return_labels=True.
    unique_labels -- The unique labels returned by DBSCAN,
                     only returned if return_labels=True.
    """

    pre = "[find_clusters_with_dbscan]:"

    if (outer_density_limit is not None):

        # The number of samples should
        # be greater than that of
        # the average density of the
        # SN in a pc^3 (~ 0.01, BT), but not as high
        # as that of an open cluster 
        # (~10 Msun / pc^3, Binney and Tremaine)

        # Note the mean number density of the solar
        # neighborhood is 0.17 stars per parsec^3,
        # while the mean in an open cluster is 17 stars
        # per parsec^3, so a good choice is the mean
        # of these in log space, or about 1 star per parsec^-3.

        # So here we are saying they should be at least closer
        # that the average distance between stars in the SN.
        number_density_limit = (outer_density_limit.value_in(u.MSun*u.parsec**-3) 
                             / avg_stellar_mass)

        if (number_density_limit < 0.17):
            print(pre, "WARNING: Your number density limit \
                        at", number_density_limit, "pc^-3 \
                        is smaller than that of the solar \
                        neighborhood, which is ~ 0.17 pc^-3!")

        eps = number_density_limit**(-1./3.)

    if (debug):
        print(pre, "outer_density_limit  =", outer_density_limit)
        print(pre, "avg_stellar_mass     =", avg_stellar_mass)
        print(pre, "number_density_limit =", number_density_limit)
        print(pre, "eps =", eps)

    particle_positions = stars.position.value_in(u.parsec)

    # Note: I don't think its necessary to scale
    #       the inputs for DBSCAN when you are
    #       using a simple Eulerian metric on 3-d
    #       position space, as its just rescaling eps.

    # Get a DBSCAN instance running.
    db = DBSCAN(eps=eps, min_samples=min_samples, leaf_size=leaf_size)
    # Do the clustering.
    clstrs = db.fit_predict(particle_positions)
    # Get the unique cluster lables (i.e. the number of clusters).
    labels = db.labels_
    unique_labels = set(labels) # This returns only the unique ones.
    # Anything with an index of -1 is noise.
    tmp = []
    for val in unique_labels:
        if val >= 0: 
            tmp.append(1)
    n_groups = len(tmp)
    #n_groups = len(filter((lambda x: x>=0),unique_labels))
    groups = []

    for label in unique_labels:
        if (label >= 0): # Don't include noise particles here.
            groups.append(stars[np.where(labels == label)[0]])

    if (debug):
        print(pre, "groups=", groups)
        print(pre, "n_groups=", n_groups)
        print(pre, "labels=", labels)
        print(pre, "unique_labels=", unique_labels)

    if (return_labels):
        return groups, n_groups, labels, unique_labels
    else:
        return groups, n_groups

In [37]:
f = h5py.File('./example_data/L3-50M-2tff_stars.amuse', 'r')
list(f.keys())
dset = f['data']
print(dset)
#find_clusters_with_dbscan(file_,debug=True)

<HDF5 group "/data" (1 members)>


In [191]:
conv = generic_unit_converter.ConvertBetweenGenericAndSiUnits(
        1.0 | u.cm, 1.0 | u.g, 1.0 | u.s)
stars = amuse.io.read_set_from_file("./example_data/L3v-2tff_stars.amuse", format='amuse')
groups, n_groups = find_clusters_with_dbscan(stars,debug=True)

[find_clusters_with_dbscan]: outer_density_limit  = 1.0 MSun * parsec**-3
[find_clusters_with_dbscan]: avg_stellar_mass     = 0.586
[find_clusters_with_dbscan]: number_density_limit = 1.70648464164
[find_clusters_with_dbscan]: eps = 0.83682093912
[find_clusters_with_dbscan]: groups= [<amuse.datamodel.particles.ParticlesSubset object at 0x122c93b10>]
[find_clusters_with_dbscan]: n_groups= 1
[find_clusters_with_dbscan]: labels= [0 0 0 ..., 0 0 0]
[find_clusters_with_dbscan]: unique_labels= {0, -1}


In [202]:
print(type(groups))
print(len(groups[0]))
print(groups[0].center_of_mass().value_in(u.m))
for group in groups:
    print(group.center_of_mass().value_in(u.m))


<class 'list'>
6247
[ -4.49123290e+16  -4.00065979e+16   9.06143981e+16]
[ -4.49123290e+16  -4.00065979e+16   9.06143981e+16]


# Blah

Now we are able to find clusters in any given AMUSE particle set. Now, need to write a function that, when given a star list constituting one or more clusters, will calculate the cluster(s)' center of mass, and the boundedness of each star to those centers of masses. Then, discard all stars that are not bound to any cluster, and return a AMUSE particle set that is once again NOT separated into clusters.

Summary of upcomming function:
1. input particle lists of clusters
2. calculate clusters' centers of mass, center of mass velocity
3. calculate boundedness of all stars to the cluster they belong to (and maybe also to other cluster com's?)
4. return a single particle set only containing remaining stars (and no separation by cluster).


In [193]:
def calculate_cluster_com_props(sg):
    group_com = []
    group_comv = []
    tmp_vel_arr = np.zeros(3)
    
    group_com.append(sg.center_of_mass().value_in(u.m))
    tmp_vel_arr[0]  = ((sg.mass*sg.vx).sum() / sg.mass.sum()).value_in(u.m / u.s)
    tmp_vel_arr[1]  = ((sg.mass*sg.vy).sum() / sg.mass.sum()).value_in(u.m / u.s)
    tmp_vel_arr[2]  = ((sg.mass*sg.vz).sum() / sg.mass.sum()).value_in(u.m / u.s)
    group_comv.append(tmp_vel_arr)
        
    return group_com, group_comv
        

In [194]:
for sg in groups:
    c,v = calculate_cluster_com_props(sg)
print(type(c[0]))
print(groups[0].position)

<class 'numpy.ndarray'>
[[-4.80286858299e+18, -3.9214214706e+18, 9.14790681815e+18], [-4.88247170306e+18, -5.50113813635e+18, 7.79473088143e+18], [-3.88436497751e+18, -8.09035613795e+18, 6.48385521913e+18], [-4.65955404486e+18, -3.48314909401e+18, 9.39936381052e+18], [-4.474335185e+18, -3.5181762901e+18, 7.63098273185e+18], [-5.06510824212e+18, -5.07759823786e+18, 8.42713861381e+18], [-5.04259647411e+18, -3.5298702274e+18, 8.9002406258e+18], [-2.37753673513e+18, -6.00224800914e+18, 1.49143519155e+19], [-3.97412574917e+18, -4.99077582505e+18, 8.94949587674e+18], [-4.58562439756e+18, -5.68846105263e+18, 8.95638651502e+18], [-4.6519860179e+18, -3.82561131245e+18, 8.11621927668e+18], [-4.64639456318e+18, -3.58115918876e+18, 1.01282125986e+19], [-3.35800538169e+18, -5.41367233248e+18, 7.76518448318e+18], [-4.45872708371e+18, -3.80942736097e+18, 9.02213410422e+18], [-1.62422066423e+18, -7.92462091508e+18, 8.17004237398e+18], [-5.11115581968e+18, -2.64917026032e+18, 8.20628218983e+18], [-4.85

TO FIND STAR BOUNDNEDNESS TO CLUSTER COM, LET'S DO ARRAY BROADCASTING! iT SHOUDL WORK, SHOUDL END WITH AN ARRAY OF BOUNDEDNESS VALUES THAT CAN THEN BBE USED TO SELECT FOR NEGATIVE VALUES THEN RETURN THE CORRESPONDING STARS

In [199]:
def calculate_star_boundedness_to_cluster_com(groups):
    G = 6.6743e-11 #m^3 kg^-2 s^-1
    total_stars = Particles()
    
    for sg in groups: # Cycle through each particle set within the list groups
        # Extract star positions, masses, velocities
        stars_xyz = sg.position.value_in(u.m)
        stars_mass = sg.mass.value_in(u.kg).reshape(len(stars_xyz),1)
        stars_vel = sg.velocity.value_in(u.m/u.s)
        stars_vel_mag = np.sqrt((sg.velocity.value_in(u.m/u.s)**2).sum(axis=1)).reshape(len(stars_xyz),1)
        # Get mass of particle set
        group_mass = sg.mass.sum().value_in(u.kg)
        # Get center of mass and com velocity of the particle set
        # and convert into len(stars)x1 array so we can do array broadcasting.
        group_com, group_comv = calculate_cluster_com_props(sg)
        com_column = np.tile(group_com,(len(stars_xyz),1))
        comv_column = np.tile(group_comv,(len(stars_xyz),1))
        
        # Broadcast arrays to get each particle's velocity relative to com velocity
        stars_vel_rel = stars_vel - comv_column
        stars_vel_rel_mag = np.sqrt((stars_vel_rel**2).sum(axis=1)).reshape(len(stars_xyz),1)
        # Same method for particle distance to com
        dist_to_com = np.sqrt(((stars_xyz-com_column)**2).sum(axis=1)).reshape(len(stars_xyz),1)

        star_KE = (0.5 * stars_mass * stars_vel_rel_mag**2).reshape(len(stars_xyz),1)
        star_PE = (-1.0 * G * group_mass * stars_mass / dist_to_com)
        star_TE = star_KE + star_PE
        
        sg_copy = sg.copy(keep_structure=True)
        unbound_star_ind = np.where(star_TE > 0)
        for ind in unbound_star_ind:
            sg_copy.remove_particle(sg_copy[ind])
        print(len(sg),len(sg_copy))
        total_stars += sg_copy

    return total_stars
calculate_star_boundedness_to_cluster_com(groups)

[array([ -4.49123290e+16,  -4.00065979e+16,   9.06143981e+16])]
6247 6094


AmuseException: Can't create new subset from particles belonging to separate particle sets. Try creating a superset instead.

In [134]:
# s = np.array([[1,1,1], [2,2,2], [3,3,3]])
# x = np.array([1,1,1])
# dist = np.zeros((len(s),1))
# print(dist)
# for i,point in enumerate(s):
#     dist[i] = np.linalg.norm(x-s[i])
# print(dist)
x1 = np.tile(x,(len(s),1))

dist = np.sqrt(((s-x1)**2).sum(axis=1)).reshape(len(s),1)
print(dist)
print(groups[0].velocity.value_in(u.m/u.s))

[[ 0.        ]
 [ 1.73205081]
 [ 3.46410162]]
[[  4.46106145e+02  -2.22569805e+03  -1.63874587e+03]
 [  3.16593353e+02  -1.90173335e+03  -1.48947132e+03]
 [  8.22018269e+01  -1.57155884e+03  -2.00612288e+03]
 [ -7.97734896e+01   1.29857706e+02  -2.10132376e+02]
 [  4.56931706e+02  -1.62166577e+03   2.23305514e+02]
 [  7.55172036e+02  -3.00750446e+02   5.34024971e+02]
 [  6.51653854e+02   5.19895813e+02   1.56925270e+03]
 [  2.04567606e+03  -1.17564311e+02   4.36323163e+02]
 [  7.91700397e+02   1.20871151e+03   1.28999370e+03]
 [  1.15787255e+03  -5.83197860e+01   1.49049167e+03]
 [  1.32672881e+03  -4.45741760e+02   2.27494879e+01]
 [  7.64345025e+02  -1.71818590e+02  -8.80417274e+02]
 [  1.57659146e+03   2.09681223e+01   9.21190384e+02]
 [  1.03075144e+03  -1.06138158e+02   9.33819248e+02]
 [  4.26742291e+02   2.17093280e+01   6.43610908e+01]
 [  1.08881788e+03   6.23652986e+02   6.16922452e+01]
 [  5.87559016e+01  -2.48336332e+01  -9.80204279e+02]
 [  3.86144618e+02   9.57338315e+02 

# Iterative bound-fraction method

[Li, H. (2019)](https://arxiv.org/pdf/1904.11987.pdf) reported using an iterative method of removing all stars from the computational domain with a positive energy:
1. Calculate total energy of all stars (Li+19 only looks at potentials from other stars since the system has evolved to be gas-less, I will need to consider the grav potential of the gas).
2. Remove all stars with positive total energy.
3. Repeat 1,2 until all stars have negative energy.

It's possible I could modify this method by also removing all gas cells that have positive energy as well.

Li+19 found that the iterative method was an appropriate way to determine the fraction of bound stars and was superior than tracking the lagrangiian radius over time for the entitre computational domain.

What I want to do:

I want to perform the iterative bound fraction method on my 2tff data outputs to compare bound fractions of stars at that point in time. If the method is quick, I'd like to do this for all of my data for each run to track the bound fraction over time. 

Then, I'd like to calculate the lagrangian radii for the entire computational domain and track over time and compare between each run and their bound fraction over time. (Figure 7 from Li+19) 

In [ ]:
# load turbsph_hdf5 plt file for 2tff

# calculate energy of each star (KE_s + PE_s + PE_g)

# note which stars have positive energy

# recalculate star energies EXCLUDING positive energy stars

# repeat